# Pivot tables with Python

Connecting to Excel:

In [1]:
import win32com.client as win32
import os

Create an Excel workbook and add a sheet

In [2]:
excel = win32.gencache.EnsureDispatch('Excel.Application')
#excel.Visible = True # Make Excel visible
workbook = excel.Workbooks.Add()
worksheet = workbook.Worksheets(1)

Adding data to the worksheet:

In [3]:
worksheet.Cells(1, 1).Value = 'Name'
worksheet.Cells(1, 2).Value = 'Category'
worksheet.Cells(1, 3).Value = 'Sales'
worksheet.Cells(2, 1).Value = 'John'
worksheet.Cells(2, 2).Value = 'Electronics'
worksheet.Cells(2, 3).Value = 1000
worksheet.Cells(3, 1).Value = 'Alice'
worksheet.Cells(3, 2).Value = 'Clothing'
worksheet.Cells(3, 3).Value = 800
worksheet.Cells(4, 1).Value = 'John'
worksheet.Cells(4, 2).Value = 'Clothing'
worksheet.Cells(4, 3).Value = 300

Selecting the data range:

In [4]:
data_range = worksheet.Range('A1:C4')

Creating a pivot table:

In [5]:
# Add a new worksheet to the workbook to hold the Pivot Table:
pivot_table_sheet = workbook.Worksheets.Add()
pivot_table_sheet.Name = 'Pivot Table'
# Create a Pivot Cache using the data range:
pivot_cache = workbook.PivotCaches().Create(SourceType=1,
SourceData=data_range)
# Create the Pivot Table on the new sheet using the Pivot Cache:
pivot_table = pivot_cache.CreatePivotTable(
 TableDestination=pivot_table_sheet.Cells(3, 1),
 TableName='MyPivotTable')
# Add the row, column and data fields
pivot_table.PivotFields('Name').Orientation = 1 # row field
pivot_table.PivotFields('Category').Orientation = 2 # columnfield
pivot_table.PivotFields('Sales').Orientation = 4 # data field
# Add the calculated fields
calculated_field = pivot_table.CalculatedFields().Add(
 "Total Sales", "=SUM(Sales)")
# Refresh the PivotTable to apply changes
pivot_table.RefreshTable()

True

Save the workbook and close Excel:

In [6]:
# Use the correct absolute path for your project
# Convert relative project path to absolute path
file_path = os.path.abspath(os.path.join('project', 'data', 'PivotTableExample.xlsx'))
print(f"Saving file to: {file_path}")

try:
    workbook.SaveAs(file_path)
    print("File saved successfully!")
except Exception as e:
    print(f"Error saving file: {e}")

    # If the first attempt fails, try an alternative approach
    try:
        # Try with double backslashes for Windows paths
        alt_path = "C:\\Lab\\Repos\\excel-with-python-and-r\\project\\data\\PivotTableExample.xlsx"
        print(f"Trying alternative path: {alt_path}")
        workbook.SaveAs(alt_path)
        print("File saved successfully with alternative path!")
    except Exception as e2:
        print(f"Alternative save also failed: {e2}")
finally:
    workbook.Close(SaveChanges=False)  # Don't try to save again if there was an error
    excel.Quit

Saving file to: C:\Lab\Repos\excel-with-python-and-r\project\project\data\PivotTableExample.xlsx
Error saving file: (-2147352567, 'Exception occurred.', (0, 'Microsoft Excel', "Microsoft Excel cannot access the file 'C:\\Lab\\Repos\\excel-with-python-and-r\\project\\project\\data\\3C728000'. There are several possible reasons:\n\n• The file name or path does not exist.\n• The file is being used by another program.\n• The workbook you are trying to save has the same name as a currently open workbook.", 'xlmain11.chm', 0, -2146827284), None)
Trying alternative path: C:\Lab\Repos\excel-with-python-and-r\project\data\PivotTableExample.xlsx
Alternative save also failed: (-2147352567, 'Exception occurred.', (0, 'Microsoft Excel', 'SaveAs method of Workbook class failed', 'xlmain11.chm', 0, -2146827284), None)


## Xlwings

In [7]:
# Using pandas for pivot tables
import xlwings as xw
import pandas as pd

All in one:

In [8]:
# Create sample data
data = {
    'Name': ['John', 'Alice', 'Bob', 'John', 'Alice', 'Bob'],
    'Category': ['Electronics', 'Clothing', 'Food', 'Clothing', 'Electronics', 'Food'],
    'Sales': [1000, 800, 750, 300, 1200, 900]
}

df = pd.DataFrame(data)

# Create a new Excel workbook
wb = xw.Book()

# Add data to the workbook
data_sheet = wb.sheets[0]
data_sheet.name = 'Data'
data_sheet.range('A1').options(index=False).value = df  # Include index=False to exclude row numbers

# Define the data range properly, ensuring it includes headers
data_range = data_sheet.range('A1').expand('table')

# Add a new sheet for the pivot table
pivot_sheet = wb.sheets.add('Pivot Table')

# Create the pivot table using a more reliable method
# First ensure Excel is properly calculating the range
data_sheet.activate()
data_address = data_range.address

# Create the pivot table
pivot_cache = wb.api.PivotCaches().Create(
    SourceType=xw.constants.PivotTableSourceType.xlDatabase,
    SourceData=data_address
)

# Create the actual pivot table
pivot_table = pivot_cache.CreatePivotTable(
    TableDestination=pivot_sheet.range('A3').api,
    TableName="SamplePivotTable"
)

# Configure the pivot table fields
pivot_table.PivotFields('Name').Orientation = xw.constants.PivotFieldOrientation.xlRowField
pivot_table.PivotFields('Category').Orientation = xw.constants.PivotFieldOrientation.xlColumnField
pivot_table.PivotFields('Sales').Orientation = xw.constants.PivotFieldOrientation.xlDataField

# Save the workbook
wb.save('data\pivot_table_examplev2.xlsx')
wb.app.quit()